In [1]:
import numpy as np
import pandas as pd
from alpharaw import register_all_readers
from alpharaw.ms_data_base import ms_reader_provider

register_all_readers()
mzml_reader = ms_reader_provider.get_reader("mzml")
mzml_reader.load_raw("../../data/raw/SYS026_RA957/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML")

/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.13/site-packages/alpharaw/raw_access/pysciexwifffilereader.py:44: UserWarning: Dotnet-based dependencies could not be loaded. Sciex support is disabled.
  warnings.warn(
/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.13/site-packages/alpharaw/raw_access/pythermorawfilereader.py:35: UserWarning: Dotnet-based dependencies could not be loaded. Thermo support is disabled.
  warnings.warn(


In [2]:
spec_df = mzml_reader.spectrum_df
spec_df

,spec_idx,peak_start_idx,peak_stop_idx,rt,precursor_mz,precursor_charge,isolation_lower_mz,isolation_upper_mz,ms_level
0,0,0,3283,0.002374,-1.0,0,-1.0,-1.0,1
1,1,3283,4654,0.006591,368.5,0,350.0,387.0,2
2,2,4654,5532,0.007747,401.0,0,386.0,416.0,2
3,3,5532,6317,0.008868,427.0,0,415.0,439.0,2
4,4,6317,7198,0.009991,450.0,0,438.0,462.0,2
...,...,...,...,...,...,...,...,...,...
89076,89076,137080673,137082379,119.995220,919.0,0,883.0,955.0,2
89077,89077,137082379,137084652,119.996410,1005.5,0,954.0,1057.0,2
89078,89078,137084652,137086933,119.998780,1353.0,0,1056.0,1650.0,2
89079,89079,137086933,137092701,120.000600,-1.0,0,-1.0,-1.0,1


In [3]:
df_diann = pd.read_parquet("../../data/raw/SYS026_RA957/DDA_SYSMHC_bynam/lib-base-result-first-pass.parquet")
df_diann = df_diann[df_diann["Run"] == "20200317_QE_HFX2_LC3_DIA_RA957_R01"]
df_diann

,Run.Index,Run,Channel,Precursor.Id,Modified.Sequence,Stripped.Sequence,Precursor.Charge,Precursor.Lib.Index,Decoy,Proteotypic,Precursor.Mz,Protein.Ids,Protein.Group,Protein.Names,Genes,RT,iRT,Predicted.RT,Predicted.iRT,IM,iIM,Predicted.IM,Predicted.iIM,Precursor.Quantity,Precursor.Normalised,Ms1.Area,Ms1.Normalised,Ms1.Apex.Area,Ms1.Apex.Mz.Delta,Normalisation.Factor,Quantity.Quality,Empirical.Quality,Normalisation.Noise,Ms1.Profile.Corr,Evidence,Mass.Evidence,Channel.Evidence,Ms1.Total.Signal.Before,Ms1.Total.Signal.After,RT.Start,...,Fr.2.Index,Fr.2.Id,Fr.2.Quantity,Fr.2.Score,Fr.3.Index,Fr.3.Id,Fr.3.Quantity,Fr.3.Score,Fr.4.Index,Fr.4.Id,Fr.4.Quantity,Fr.4.Score,Fr.5.Index,Fr.5.Id,Fr.5.Quantity,Fr.5.Score,Fr.6.Index,Fr.6.Id,Fr.6.Quantity,Fr.6.Score,Fr.7.Index,Fr.7.Id,Fr.7.Quantity,Fr.7.Score,Fr.8.Index,Fr.8.Id,Fr.8.Quantity,Fr.8.Score,Fr.9.Index,Fr.9.Id,Fr.9.Quantity,Fr.9.Score,Fr.10.Index,Fr.10.Id,Fr.10.Quantity,Fr.10.Score,Fr.11.Index,Fr.11.Id,Fr.11.Quantity,Fr.11.Score
1,1,20200317_QE_HFX2_LC3_DIA_RA957_R01,,(UniMod:1)AAAAVVEFQR2,(UniMod:1)AAAAVVEFQR,AAAAVVEFQR,2,0,0,1,552.295776,2/sp|O00231|PSD11_HUMAN/sp|O00231-2|PSD11_HUMAN,2/sp|O00231|PSD11_HUMAN/sp|O00231-2|PSD11_HUMAN,,,90.442459,75.985481,91.231018,75.398308,0.0,0.0,0.0,0.0,9.424966e+05,8.926844e+05,6.395898e+06,6.057866e+06,2.873886e+06,0.000488,0.947149,0.561699,0.0,0.034040,0.412459,1.970261,0.0,0.567322,6.335309e+09,5.958802e+09,90.164604,...,3,?0^1/327.167603,2.616499e+05,0.257604,4,?0^1/777.427673,1.710286e+05,0.477368,5,?0^1/848.465210,2.518822e+05,0.602471,6,?0^1/579.289673,6.949832e+04,0.266569,8,?0^1/450.248505,3.321095e+04,0.135435,10,?0^1/525.304688,1.065813e+04,-0.018717,11,?0^1/919.490417,0.000000,0.000000,-1,,0.0000,0.000000,-1,,0.0,0.000000,-1,,0.000000,0.000000
3,1,20200317_QE_HFX2_LC3_DIA_RA957_R01,,(UniMod:1)AAGIVASR2,(UniMod:1)AAGIVASR,AAGIVASR,2,1,0,1,393.727112,1/sp|Q13011|ECH1_HUMAN,1/sp|Q13011|ECH1_HUMAN,,,59.042984,49.335880,59.132977,49.304188,0.0,0.0,0.0,0.0,6.859579e+07,6.063233e+07,2.902776e+08,2.565785e+08,2.727381e+08,-0.000031,0.883907,0.973036,0.0,0.008771,0.991708,6.567415,0.0,0.966696,1.599040e+10,1.918689e+10,58.919956,...,3,?0^1/333.188812,1.662300e+07,0.975289,6,?0^1/602.362610,2.124371e+07,0.969861,7,?0^1/245.125900,2.686377e+06,0.899896,8,?0^1/262.152802,3.955332e+06,0.974825,9,?0^1/355.199005,4.302696e+06,0.965792,10,?0^1/545.342224,1.156535e+07,0.573526,-1,,0.000000,0.000000,-1,,0.0000,0.000000,-1,,0.0,0.000000,-1,,0.000000,0.000000
5,1,20200317_QE_HFX2_LC3_DIA_RA957_R01,,(UniMod:1)ASNKTTLQK2,(UniMod:1)ASNKTTLQK,ASNKTTLQK,2,2,0,1,516.787781,1/sp|Q13185|CBX3_HUMAN,1/sp|Q13185|CBX3_HUMAN,,,24.648211,21.576439,25.976175,20.409519,0.0,0.0,0.0,0.0,2.487642e+06,2.419243e+06,5.809768e+06,5.650026e+06,4.048920e+06,0.000488,0.972505,0.796623,0.0,-0.024936,0.469674,1.385189,0.0,0.599972,1.188404e+10,1.197718e+10,24.554205,...,5,?0^2/460.265503,0.000000e+00,0.000000,6,?0^1/590.352722,0.000000e+00,0.000000,8,?0^1/240.134903,2.217324e+06,0.651661,9,?0^1/275.172089,7.219640e+05,0.219825,10,?0^1/257.161499,5.893810e+05,0.502722,11,?0^1/718.450012,0.000000e+00,0.000000,-1,,0.000000,0.000000,-1,,0.0000,0.000000,-1,,0.0,0.000000,-1,,0.000000,0.000000
6,1,20200317_QE_HFX2_LC3_DIA_RA957_R01,,(UniMod:1)MERFVVTAPPARNR3,(UniMod:1)MERFVVTAPPARNR,MERFVVTAPPARNR,3,5,0,1,562.635010,1/sp|Q9UFW8|CGBP1_HUMAN,1/sp|Q9UFW8|CGBP1_HUMAN,,,76.139687,63.269024,75.752525,63.590595,0.0,0.0,0.0,0.0,1.019322e+06,9.707749e+05,5.428902e+06,5.170338e+06,4.230866e+06,-0.000549,0.952373,-0.309303,0.0,-0.013395,0.865326,1.505997,0.0,0.000000,1.647940e+10,1.514525e+10,76.047752,...,4,?0^2/284.160706,6.940294e+05,0.035142,5,?0^2/355.706696,8.174430e+04,0.841770,6,?0^2/201.123596,7.601060e+06,-0.380853,7,?0^2/298.176086,1.319377e+06,0.489120,8,?0^2/215.139099,3.357117e+06,-0.388600,9,?0^1/976.494873,0.000000e+00,0.000000,10,?0^1/289.162292,37242.945312,-0.041680,-1,,0.0000,0.000000,-1,,0.0,0.000000,-1,,0.000000,0.000000
9,1,20200317_QE_HFX2_LC3_DIA_RA957_R01,,(UniMod:1)S

In [4]:
%%time
# test no jit function call
import pandas as pd


def find_DIA_spec_idx_by_rt_nojit(
    spec_rts: np.ndarray,
    spec_isolation_lower_mzs: np.ndarray,
    spec_isolation_upper_mzs: np.ndarray,
    query_rt: float,
    query_precursor_mz: float,
    rt_tolerance: float = -1.0,
) -> np.int32:
    rt_idx = np.searchsorted(spec_rts, query_rt)
    if rt_idx == 0:
        candidates = [0, 1] if len(spec_rts) > 1 else [0]
    elif rt_idx >= len(spec_rts):
        candidates = [len(spec_rts) - 2, len(spec_rts) - 1] if len(spec_rts) > 1 else [len(spec_rts) - 1]
    else:
        left_idx = rt_idx - 1
        right_idx = rt_idx
        left_dist = abs(spec_rts[left_idx] - query_rt)
        right_dist = abs(spec_rts[right_idx] - query_rt)

        if left_dist <= right_dist:
            candidates = [left_idx, right_idx]
        else:
            candidates = [right_idx, left_idx]

    for idx in candidates:
        if idx < 0 or idx >= len(spec_rts):
            continue
        # Check if isolation window contains precursor m/z
        if spec_isolation_lower_mzs[idx] <= query_precursor_mz <= spec_isolation_upper_mzs[idx]:
            # Check RT tolerance
            if abs(spec_rts[idx] - query_rt) > rt_tolerance:
                continue

            return np.int32(idx)

    return np.int32(-1)


spec_rts = spec_df["rt"].to_numpy()
spec_isolation_lower_mzs = spec_df["isolation_lower_mz"].to_numpy()
spec_isolation_upper_mzs = spec_df["isolation_upper_mz"].to_numpy()
fail_count = 0
for _idx, row in df_diann.iterrows():
    query_rt = row["RT"]
    query_precursor_mz = row["Precursor.Mz"]
    spec_idx = find_DIA_spec_idx_by_rt_nojit(
        spec_rts,
        spec_isolation_lower_mzs,
        spec_isolation_upper_mzs,
        query_rt,
        query_precursor_mz,
        rt_tolerance=1e-4,  # in minutes, means 166.67 Hz
    )
    if spec_idx == -1:
        fail_count += 1
    # print(f"Peptide: {row['Modified.Sequence']}, RT: {query_rt}, PrecursorMz: {query_precursor_mz} => SpecIdx: {spec_idx}")

print(f"Total failed to find spectra: {fail_count} out of {len(df_diann)}")

Total failed to find spectra: 0 out of 169226
CPU times: user 1.91 s, sys: 54.1 ms, total: 1.97 s
Wall time: 1.97 s


In [5]:
%%time
# test no jit function call
import numba
import numpy as np
import pandas as pd


@numba.njit
def find_DIA_spec_idx_by_rt_jit(
    spec_rts: np.ndarray,
    spec_isolation_lower_mzs: np.ndarray,
    spec_isolation_upper_mzs: np.ndarray,
    query_rt: float,
    query_precursor_mz: float,
    rt_tolerance: float = -1.0,
) -> np.int32:
    rt_idx = np.searchsorted(spec_rts, query_rt)
    if rt_idx == 0:
        candidates = [0, 1] if len(spec_rts) > 1 else [0]
    elif rt_idx >= len(spec_rts):
        candidates = [len(spec_rts) - 2, len(spec_rts) - 1] if len(spec_rts) > 1 else [len(spec_rts) - 1]
    else:
        left_idx = rt_idx - 1
        right_idx = rt_idx
        left_dist = abs(spec_rts[left_idx] - query_rt)
        right_dist = abs(spec_rts[right_idx] - query_rt)

        if left_dist <= right_dist:
            candidates = [left_idx, right_idx]
        else:
            candidates = [right_idx, left_idx]

    for idx in candidates:
        if idx < 0 or idx >= len(spec_rts):
            continue
        # Check if isolation window contains precursor m/z
        if spec_isolation_lower_mzs[idx] <= query_precursor_mz <= spec_isolation_upper_mzs[idx]:
            # Check RT tolerance
            if abs(spec_rts[idx] - query_rt) > rt_tolerance:
                continue

            return np.int32(idx)

    return np.int32(-1)


spec_rts = spec_df["rt"].to_numpy()
spec_isolation_lower_mzs = spec_df["isolation_lower_mz"].to_numpy()
spec_isolation_upper_mzs = spec_df["isolation_upper_mz"].to_numpy()
fail_count = 0
for _idx, row in df_diann.iterrows():
    query_rt = row["RT"]
    query_precursor_mz = row["Precursor.Mz"]
    spec_idx = find_DIA_spec_idx_by_rt_jit(
        spec_rts,
        spec_isolation_lower_mzs,
        spec_isolation_upper_mzs,
        query_rt,
        query_precursor_mz,
        rt_tolerance=1e-4,  # in minutes, means 166.67 Hz
    )
    if spec_idx == -1:
        fail_count += 1
    # print(f"Peptide: {row['Modified.Sequence']}, RT: {query_rt}, PrecursorMz: {query_precursor_mz} => SpecIdx: {spec_idx}")

print(f"Total failed to find spectra: {fail_count} out of {len(df_diann)}")

Total failed to find spectra: 0 out of 169226
CPU times: user 2.04 s, sys: 127 ms, total: 2.17 s
Wall time: 2.31 s


In [6]:
%%time
from dia_aspire_rescore.psm.spec_finder import find_single_DIA_spec_idx_by_rt

spec_rts = spec_df["rt"].to_numpy()
spec_isolation_lower_mzs = spec_df["isolation_lower_mz"].to_numpy()
spec_isolation_upper_mzs = spec_df["isolation_upper_mz"].to_numpy()
fail_count = 0
for _idx, row in df_diann.iterrows():
    query_rt = row["RT"]
    query_precursor_mz = row["Precursor.Mz"]
    spec_idx = find_single_DIA_spec_idx_by_rt(
        spec_rts,
        spec_isolation_lower_mzs,
        spec_isolation_upper_mzs,
        query_rt,
        query_precursor_mz,
        rt_tolerance=1e-4,  # in minutes, means 166.67 Hz
    )
    if spec_idx == -1:
        fail_count += 1
    # print(f"Peptide: {row['Modified.Sequence']}, RT: {query_rt}, PrecursorMz: {query_precursor_mz} => SpecIdx: {spec_idx}")

print(f"Total failed to find spectra: {fail_count} out of {len(df_diann)}")

Total failed to find spectra: 0 out of 169226
CPU times: user 2.95 s, sys: 201 ms, total: 3.15 s
Wall time: 3.26 s


In [7]:
%%time
from dia_aspire_rescore.psm.spec_finder import find_batch_DIA_spec_idxes_by_rt

spec_rts = spec_df["rt"].to_numpy()
spec_isolation_lower_mzs = spec_df["isolation_lower_mz"].to_numpy()
spec_isolation_upper_mzs = spec_df["isolation_upper_mz"].to_numpy()
query_rts = df_diann["RT"].to_numpy()
query_precursor_mzs = df_diann["Precursor.Mz"].to_numpy()
spec_idxes = find_batch_DIA_spec_idxes_by_rt(
    spec_rts,
    spec_isolation_lower_mzs,
    spec_isolation_upper_mzs,
    query_rts,
    query_precursor_mzs,
    rt_tolerance=1e-4,  # in minutes, means 166.67 Hz
)
fail_count = np.sum(spec_idxes == -1)
print(f"Total failed to find spectra (batch): {fail_count} out of {len(df_diann)}")

Total failed to find spectra (batch): 0 out of 169226
CPU times: user 257 ms, sys: 7.87 ms, total: 265 ms
Wall time: 267 ms


In [8]:
%%time
from dia_aspire_rescore.psm.spec_finder import find_DIA_spec_idxes_by_rt

spec_idxes = find_DIA_spec_idxes_by_rt(
    spec_df,
    df_diann["RT"].to_numpy(),
    df_diann["Precursor.Mz"].to_numpy(),
    rt_tolerance=1e-4,  # in minutes, means 166.67 Hz
)
fail_count = np.sum(spec_idxes == -1)
print(f"Total failed to find spectra (batch via spec_df): {fail_count} out of {len(df_diann)}")

Total failed to find spectra (batch via spec_df): 0 out of 169226
CPU times: user 24.4 ms, sys: 733 μs, total: 25.1 ms
Wall time: 24.8 ms


In [ ]:
from alpharaw.match.spec_finder import find_dia_spec_idxes_same_window

spec_rt_values = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
query_rt_values = np.array([0.5, 3.0, 6.0])  
max_spec_per_query = 3

result = find_dia_spec_idxes_same_window(
    spec_rt_values, query_rt_values, max_spec_per_query
)
# this method is out of bound 
result

array([[0, 1, 2, 3],
       [2, 3, 4, 5],
       [3, 4, 5, 6]], dtype=int32)

In [16]:
spec_rt_values = np.array([0,1,2,3,4,5,6,7,8,9,10])
query_rt_values = np.array([9.5])  
max_spec_per_query = 5

result = find_dia_spec_idxes_same_window(
    spec_rt_values, query_rt_values, max_spec_per_query
)
# this method is out of bound 
result

array([[ 8,  9, 10, 11, 12]], dtype=int32)